# 5. Content Classification Demo

Demo notebook để phân loại nội dung OCR output và vẽ bounding boxes.

## Các loại nội dung được phân loại:
- **TABLE** (Đỏ): Bảng dữ liệu với các cột và hàng
- **FORM** (Xanh dương): Form với các cặp key:value
- **FIGURE** (Xanh lá): Chart, plot, biểu đồ
- **TEXT** (Cam): Đoạn văn bản thông thường

## Output:
- Ảnh với bounding boxes được tô màu theo loại nội dung
- JSON file với classification results
- Comparison view và Type grid

In [ ]:
import sys
sys.path.insert(0, '..')

import os
import glob
import matplotlib.pyplot as plt
from PIL import Image

# Import modules
from src.ocr import (
    PaddleOCRProcessor,
    ContentClassifier,
    ContentType,
    BBoxVisualizer,
    visualize_classification,
    create_demo_output
)

print("✓ Modules imported successfully")

## 1. Khởi tạo OCR Processor

In [ ]:
# Khởi tạo processor
processor = PaddleOCRProcessor()
print("✓ OCR Processor initialized")

## 2. Tìm ảnh mẫu

In [ ]:
# Tìm ảnh mẫu trong dataset
image_paths = glob.glob('../dataset/DocVQA_Images/**/*.png', recursive=True)
if not image_paths:
    image_paths = glob.glob('../dataset/**/*.png', recursive=True)
if not image_paths:
    image_paths = glob.glob('../**/*.png', recursive=True)

if image_paths:
    sample_image = image_paths[0]
    print(f"Ảnh mẫu: {sample_image}")
else:
    sample_image = None
    print("⚠️ Không tìm thấy ảnh mẫu. Vui lòng chỉ định đường dẫn ảnh.")

## 3. Chạy OCR và Phân loại nội dung

In [ ]:
if sample_image:
    # Chạy OCR với phân loại nội dung
    ocr_result, classification = processor.run_ocr_with_classification(
        sample_image,
        use_preprocessing=True
    )
    
    print(f"\n{'='*60}")
    print("KẾT QUẢ OCR")
    print(f"{'='*60}")
    print(f"Số dòng phát hiện: {ocr_result['num_lines']}")
    print(f"Độ tin cậy TB: {ocr_result['avg_confidence']:.2%}")
    
    print(f"\n{'='*60}")
    print("KẾT QUẢ PHÂN LOẠI")
    print(f"{'='*60}")
    print(f"Tables (Bảng):    {classification.summary.get('table', 0)}")
    print(f"Forms (Key:Value): {classification.summary.get('form', 0)}")
    print(f"Figures (Chart):  {classification.summary.get('figure', 0)}")
    print(f"Text (Paragraph): {classification.summary.get('text', 0)}")
    print(f"{'='*60}")

## 4. Hiển thị chi tiết các regions

In [ ]:
if sample_image and classification.regions:
    print(f"\n{'='*80}")
    print("CHI TIẾT CÁC REGIONS")
    print(f"{'='*80}")
    
    for region in classification.regions:
        print(f"\n[Region {region.region_id}] {region.content_type.value.upper()}")
        print(f"  Confidence: {region.confidence:.3f}")
        print(f"  Lines: {len(region.lines)}")
        
        # In metadata theo loại
        if region.content_type == ContentType.TABLE:
            print(f"  Columns: {region.metadata.get('num_columns', 0)}")
            print(f"  Column Stability: {region.metadata.get('column_stability', 0):.2f}")
        elif region.content_type == ContentType.FORM:
            print(f"  Key:Value Pattern: {region.metadata.get('has_keyvalue_pattern', False)}")
            print(f"  Colon Ratio: {region.metadata.get('colon_ratio', 0):.2f}")
        elif region.content_type == ContentType.FIGURE:
            print(f"  Empty Center: {region.metadata.get('empty_center_ratio', 0):.2f}")
            print(f"  Has Legend: {region.metadata.get('has_legend', False)}")
        elif region.content_type == ContentType.TEXT:
            print(f"  Avg Line Length: {region.metadata.get('avg_line_length', 0):.1f}")
            print(f"  Spacing Uniformity: {region.metadata.get('spacing_uniformity', 0):.2f}")
        
        # Preview text content
        if region.text_content:
            preview = region.text_content[:100].replace('\n', ' ')
            print(f"  Preview: {preview}..." if len(region.text_content) > 100 else f"  Preview: {preview}")

## 5. Visualization - Bounding Boxes với màu theo loại

In [ ]:
if sample_image and classification.regions:
    # Vẽ bounding boxes
    img_classified = processor.draw_classified_regions(
        sample_image,
        classification,
        fill=True
    )
    
    # Hiển thị
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    
    # Ảnh gốc
    axes[0].imshow(Image.open(sample_image))
    axes[0].set_title('Ảnh gốc', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Ảnh với classification
    axes[1].imshow(img_classified)
    axes[1].set_title('Content Classification\n(Red=Table, Blue=Form, Green=Figure, Orange=Text)', 
                      fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

## 6. Comparison View

In [ ]:
if sample_image and classification.regions:
    # Tạo comparison view
    visualizer = BBoxVisualizer()
    comparison_img = visualizer.create_comparison_view(sample_image, classification)
    
    plt.figure(figsize=(24, 10))
    plt.imshow(comparison_img)
    plt.title('Comparison: Original → OCR Detected → Content Classified', fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 7. Type Grid View

In [ ]:
if sample_image and classification.regions:
    # Tạo type grid
    type_grid_img = visualizer.create_type_grid(sample_image, classification)
    
    plt.figure(figsize=(20, 16))
    plt.imshow(type_grid_img)
    plt.title('Content Type Grid', fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 8. Lọc theo loại nội dung cụ thể

In [ ]:
if sample_image:
    # Chỉ hiển thị tables và forms
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    
    # Chỉ Tables
    img_tables = processor.visualize_content_types(
        sample_image,
        ocr_result,
        content_types=[ContentType.TABLE]
    )
    axes[0].imshow(img_tables)
    axes[0].set_title(f'Chỉ TABLES (Đỏ)\n({classification.summary.get("table", 0)} detected)', 
                      fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Chỉ Forms
    img_forms = processor.visualize_content_types(
        sample_image,
        ocr_result,
        content_types=[ContentType.FORM]
    )
    axes[1].imshow(img_forms)
    axes[1].set_title(f'Chỉ FORMS (Xanh dương)\n({classification.summary.get("form", 0)} detected)', 
                      fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

## 9. Trích xuất nội dung theo loại

In [ ]:
if classification and classification.regions:
    # Lấy tất cả tables
    tables = classification.get_tables()
    print(f"\n{'='*60}")
    print(f"TABLES ({len(tables)} found)")
    print(f"{'='*60}")
    for i, table in enumerate(tables, 1):
        print(f"\nTable {i} (confidence: {table.confidence:.3f})")
        print("-" * 40)
        for line in table.lines[:5]:  # Preview 5 dòng đầu
            print(f"  {line}")
        if len(table.lines) > 5:
            print(f"  ... ({len(table.lines) - 5} more lines)")
    
    # Lấy tất cả forms
    forms = classification.get_forms()
    print(f"\n{'='*60}")
    print(f"FORMS ({len(forms)} found)")
    print(f"{'='*60}")
    for i, form in enumerate(forms, 1):
        print(f"\nForm {i} (confidence: {form.confidence:.3f})")
        print("-" * 40)
        for line in form.lines[:5]:
            print(f"  {line}")
        if len(form.lines) > 5:
            print(f"  ... ({len(form.lines) - 5} more lines)")

## 10. Export Demo Output

In [ ]:
if sample_image:
    # Tạo bộ demo outputs đầy đủ
    output_dir = '../demo_output'
    
    results = processor.create_demo_visualization(
        sample_image,
        ocr_result=ocr_result,
        output_dir=output_dir
    )
    
    # Hiển thị paths
    if results['output_paths']:
        print(f"\n✅ Demo outputs saved:")
        for key, path in results['output_paths'].items():
            print(f"   - {key}: {path}")

## 11. Quick API Demo

In [ ]:
# Quick API - One-liner classification
if sample_image:
    from src.ocr import classify_ocr_output, visualize_classification
    
    # Quick classify
    quick_classification = classify_ocr_output(ocr_result)
    print(f"Quick Classification Summary: {quick_classification.summary}")
    
    # Quick visualize
    img, result = visualize_classification(
        sample_image,
        ocr_result,
        output_path='../demo_output/quick_viz.png',
        show=True
    )

## Legend

| Color | Content Type | Description |
|-------|--------------|-------------|
| 🔴 Red | TABLE | Bảng dữ liệu với các cột và hàng |
| 🔵 Blue | FORM | Form với các cặp key:value |
| 🟢 Green | FIGURE | Chart, plot, biểu đồ |
| 🟠 Orange | TEXT | Đoạn văn bản thông thường |